In [10]:
# As usual, a bit of setup
import caffe 
import pickle
import numpy as np

## Load model and input image dir

In [19]:
! ls caffe/dl.caffe.berkeleyvision.org/

bvlc_alexnet.caffemodel  deploy.prototxt


In [25]:
# required only if working in gpu mode
gpu_id = 0

extract_from_layer = "fc7"
input_images_file = "caffe/small-image-set/"
model_def= "caffe/dl.caffe.berkeleyvision.org/deploy.prototxt"
pretrained_model="caffe/dl.caffe.berkeleyvision.org/bvlc_alexnet.caffemodel"

# output file to write extracted features to disk
output_pkl_file_name = "caffe/output_features.pkl"

# change based on your deploy.prototxt file
batch_size = 10

In [27]:
input_exp_file = 'caffe/small_set.csv'

In [28]:
ext_file = open(input_exp_file, 'r')
image_paths_list = [line.strip() for line in ext_file]
ext_file.close()

In [29]:
caffe.set_mode_gpu()
caffe.set_device(gpu_id)

In [30]:
images_loaded_by_caffe = [caffe.io.load_image(im) for im in image_paths_list] 

### Image transformer

In [31]:
# set up transformer - creates transformer object
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

# transpose image from HxWxC to CxHxW
transformer.set_transpose('data', (2,0,1))

# swap image channels from RGB to BGR
transformer.set_channel_swap('data', (2,1,0))

# set raw_scale = 255 to multiply with the values loaded with caffe.io.load_image
transformer.set_raw_scale('data', 255)

### Load DAG

In [32]:
net = caffe.Net(model_def, pretrained_model, caffe.TEST)

### Grab a batch of images

In [33]:
def get_this_batch(image_list, batch_index, batch_size):
    start_index = batch_index * batch_size
    next_batch_size = batch_size    
    image_list_size = len(image_list)
    # batches might not be evenly divided
    if(start_index + batch_size > image_list_size):  
        reamaining_size_at_last_index = image_list_size - start_index
        next_batch_size = reamaining_size_at_last_index
    batch_index_indices = range(start_index, start_index+next_batch_size,1)
    return image_list[batch_index_indices]

In [35]:
total_batch_nums = len(images_loaded_by_caffe) // batch_size    
features_all_images = []
images_loaded_by_caffe = np.array(images_loaded_by_caffe)

# loop through all the batches 
for j in range(total_batch_nums+1):
	image_batch_to_process = get_this_batch(images_loaded_by_caffe, j, batch_size)
	num_images_being_processed = len(image_batch_to_process)
	data_blob_index = range(num_images_being_processed)

	# note that each batch is passed through a transformer 
	# before passing to data layer
	net.blobs['data'].data[data_blob_index] = [transformer.preprocess('data', img) for img in image_batch_to_process]

	# BEWARE: blobs arrays are overwritten
	res = net.forward()

	# actual batch feature extraction
	features_for_this_batch = net.blobs[extract_from_layer].data[data_blob_index].copy()
	features_all_images.extend(features_for_this_batch)

/home/ubuntu/anaconda3/envs/caffe_p35/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [36]:
pkl_object = {"filename": image_paths_list, "features": features_all_images}
output = open(output_pkl_file_name, 'wb')
pickle.dump(pkl_object, output, 2)
output.close()